<a href="https://colab.research.google.com/github/Naiker911/Hybrid-recommendation-systems/blob/main/%D0%94_%D0%97_%C2%AB%D0%93%D0%B8%D0%B1%D1%80%D0%B8%D0%B4%D0%BD%D1%8B%D0%B5_%D1%80%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D1%8B%C2%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задание:

1. Датасет ml-latest
2. Вспомнить подходы, которые мы разбирали
3. Выбрать понравившийся подход к гибридным системам
4. Написать свою

In [ ]:
!pip install lightfm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=879174 sha256=432a66ef0c777080d7db38d132aa47d1cfa83a001d8bfef3c09cf510f4afc014
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [ ]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 20.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095408 sha256=b01501ef37779e5dcc191de4d93bb4b5f7a2b47ceb2d162e011f80f9cc335e9a
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

import pandas as pd
import numpy as np

In [ ]:
!wget 'https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx' -O MovieLens.zip

--2023-06-22 09:06:55--  https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.google.com (drive.google.com)... 142.251.31.100, 142.251.31.113, 142.251.31.139, ...
Connecting to drive.google.com (drive.google.com)|142.251.31.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0g-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ur32214ft251cdi5t8u2qtmdrpnu2h7d/1687424775000/02611596255248067438/*/1m0rwReR09achL0xTM6QPoN4tykz5bOMx?uuid=0cac890b-e75b-45c0-856e-de2240aa3a57 [following]
--2023-06-22 09:06:56--  https://doc-0g-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ur32214ft251cdi5t8u2qtmdrpnu2h7d/1687424775000/02611596255248067438/*/1m0rwReR09achL0xTM6QPoN4tykz5bOMx?uuid=0cac890b-e75b-45c0-856e-de2240aa3a57
Resolving doc-0g-84-docs.googleusercontent.com (doc-0g-84-docs.googleusercontent.com)... 108.177.126.132, 2a00:1450:4013:c01::84
Connecting to doc-0

In [ ]:
!unzip MovieLens.zip

Archive:  MovieLens.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: tags.csv                


In [ ]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [ ]:
movies_with_ratings = movies.merge(ratings, on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [ ]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [ ]:
reader = Reader(rating_scale=(ratings.rating.min(), ratings.rating.max()))
data = Dataset.load_from_df(dataset, reader)

In [ ]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=21)

Обучение RecSys из библиотеки surprise

В качестве модели выбрана SVD библиотеки surprise: пожертвуем немного качеством модели по сравнению с SVDpp, но кратно сократим время.

In [ ]:
svd = SVD(n_factors=100, n_epochs=50, lr_all=0.01, reg_all=0.08, random_state=21)
svd.fit(trainset)

In [ ]:
predictions = svd.test(testset)
accur_score = accuracy.rmse(predictions, verbose=True)

RMSE: 0.8519


In [ ]:
def get_recommend(user_id):
    'Предоставление рекомендаций фильмов по userID'
    user_movies = movies_with_ratings[movies_with_ratings.userId == user_id].title.unique()
    scores = []
    titles = []
    for movie in movies_with_ratings.title.unique():
        if movie in user_movies:
            continue

        scores.append(svd.predict(uid=user_id, iid=movie).est)
        titles.append(movie)
    titles = np.array(titles)
    return titles[np.argsort(-np.array(scores))[:10]]

In [ ]:
userId = 36
print(f'Для пользователя с ID {userId} рекомендуются следующие фильмы:')
print('--------------')
print(*get_recommend(userId), sep='\n')

Для пользователя с ID 36 рекомендуются следующие фильмы:
--------------
Neon Genesis Evangelion: The End of Evangelion (Shin seiki Evangelion Gekijô-ban: Air/Magokoro wo, kimi ni) (1997)
Celebration, The (Festen) (1998)
Godfather, The (1972)
Bad Boy Bubby (1993)
The Artist (2011)
Goodfellas (1990)
Paths of Glory (1957)
Wristcutters: A Love Story (2006)
Black Mirror: White Christmas (2014)
Mean Streets (1973)


Обучение RecSys на основе content-based с применением TF-IDF

В качестве модели content-based модель с применением TF-IDF к признаку genre. Рекомендации посчитаем с помощью метода NearestNeighbors.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelEncoder

from scipy.sparse import coo_matrix

from surprise import accuracy
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV

In [ ]:
def get_X_y(data):
    tfidf = TfidfVectorizer()
    data['genres'] = data['genres'].apply(lambda val: val.replace('|', ' '))
    X = pd.DataFrame(tfidf.fit_transform(data['genres']).toarray(),
                       columns=tfidf.get_feature_names_out()).reset_index(drop=True)
    return X, data['rating']

In [ ]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [ ]:
movie_genres = [change_string(g) for g in movies.genres.values]

In [ ]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(movie_genres)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

neigh = NearestNeighbors(n_neighbors=20, n_jobs=-1, metric='euclidean')
neigh.fit(X_train_tfidf)

NearestNeighbors(metric='euclidean', n_jobs=-1, n_neighbors=20)

Добавим дополнительный признак для фильмов: их средний рейтинг. После подбора похожих фильмов, отсортируем их по более рейтинговым

In [ ]:
mean_movie_rating = movies_with_ratings.groupby('movieId')['rating'].mean()
movies_with_ratings['mean_movie_rating'] = movies_with_ratings['movieId'].apply(lambda x: mean_movie_rating[x])

In [ ]:
def get_rec_by_genres(genres):
    predict = count_vect.transform([genres])
    X_tfidf2 = tfidf_transformer.transform(predict)
    res = neigh.kneighbors(X_tfidf2, return_distance=True)
    return np.array(movies_with_ratings.iloc[res[1][0]].sort_values(by=['mean_movie_rating'], ascending=False)['title'].unique()[:10])

In [ ]:
genres = 'Adventure Comedy Fantasy Crime'
print(f'По жанрам "{genres}" рекомендуются следующие фильмы:')
print('--------------')
print(*get_rec_by_genres(genres), sep='\n')

По жанрам "Adventure Comedy Fantasy Crime" рекомендуются следующие фильмы:
--------------
Pulp Fiction (1994)
Braveheart (1995)
Three Colors: White (Trzy kolory: Bialy) (1994)
Seven (a.k.a. Se7en) (1995)
Antonia's Line (Antonia) (1995)
Bullets Over Broadway (1994)
Disclosure (1994)
Mighty Aphrodite (1995)
Interview with the Vampire: The Vampire Chronicles (1994)
Happy Gilmore (1996)


Гибридная RecSys

Объединяет в себе рекомендательную систему на основе content-based и SVD библиотеки surprise

In [ ]:
movies_with_ratings.sort_values('timestamp', inplace=True)

In [ ]:
title_genres = {}

for index, row in movies.iterrows():
    title_genres[row.title] = row.genres

In [ ]:
def recommend_for_user_by_genre(user_id):
    user_movies = movies_with_ratings[movies_with_ratings.userId == user_id].title.unique()

    last_user_movie = user_movies[-1]

    movie_genres = title_genres[last_user_movie]

    movie_genres = change_string(movie_genres)

    predict = count_vect.transform([movie_genres])

    X_tfidf2 = tfidf_transformer.transform(predict)

    res = neigh.kneighbors(X_tfidf2, return_distance=True)

    movies_to_score = movies.iloc[res[1][0]].title.values

    scores = []
    titles = []

    for movie in movies_to_score:
        if movie in user_movies:
            continue

        scores.append(svd.predict(uid=user_id, iid=movie).est)
        titles.append(movie)

    best_indexes = np.argsort(scores)[10:]

    for i in reversed(best_indexes):
        print(titles[i], scores[i])

In [ ]:
userId = 36
print(f'Для пользователя с ID {userId} рекомендуются следующие фильмы:')
print('--------------')
recommend_for_user_by_genre(userId)

Для пользователя с ID 36 рекомендуются следующие фильмы:
--------------
Godfather, The (1972) 3.9065630536696836
Goodfellas (1990) 3.8388909757441
Godfather: Part II, The (1974) 3.69341504201646
Heavenly Creatures (1994) 3.495484104803821
Bonnie and Clyde (1967) 3.395729024775608
Hate (Haine, La) (1995) 3.3675678590832083
Jason's Lyric (1994) 3.318962708680239
Once Upon a Time in America (1984) 3.2939734661859155
Casino (1995) 3.2906532028543394


RecSys на основе библиотеки Lightfm

In [ ]:
from lightfm import LightFM
import lightfm.data as light
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

In [ ]:
def create_rate_matrix(df, shuffle = True, split_ratio = 0.8):
    '''
    Split the Pandas DataFrame into train and test according to the split_ratio.
    INPUT:
      - df: Pandas DataFrame of interaction data, including user id, product id, and rate.
      - shuffle: boolean, whether to randomly shuffle the dataframe before splitting
      - split_ratio: the ratio of train and test
    OUTPUT:
      - rate_matrix: a dictionary, keys ['train', 'test'], value is coo_matrix of the same shape
    '''
    if shuffle:
        df = df.sample(frac = 1).reset_index(drop = True)
    split_point = np.int(np.round(df.shape[0] * split_ratio))
    df_train = df.iloc[0:split_point]
    df_test = df.iloc[split_point::]
    df_test = df_test[(df_test['userId'].isin(df_train['userId']))&\
                     (df_test['movieId'].isin(df_train['movieId']))]

    print('Train dataset size is %d, test dataset size is %d'
          % (len(df_train), len(df_test)))

    id_cols = ['userId', 'movieId']
    trans_cat_train = dict()
    trans_cat_test = dict()

    encoder = dict()
    for k in id_cols:
        le = LabelEncoder()
        trans_cat_train[k] = le.fit_transform(df_train[k].values)
        trans_cat_test[k] = le.transform(df_test[k].values)
        encoder[k] = le

    trans_cat_train['rating'] = df_train['rating']
    trans_cat_test['rating'] = df_test['rating']

    users = np.unique(trans_cat_train['userId'])
    items = np.unique(trans_cat_train['movieId'])
    n_users = len(users)
    n_items = len(items)
    print('There are %d users and %d products in dataset.'
          % (n_users, n_items))

    rate_matrix = dict()
    rate_matrix['train'] = coo_matrix((trans_cat_train['rating'],
                                       (trans_cat_train['userId'],
                                        trans_cat_train['movieId'])),
                                      shape = (n_users, n_items))

    rate_matrix['test'] = coo_matrix((trans_cat_test['rating'],
                                      (trans_cat_test['userId'],
                                       trans_cat_test['movieId'])),
                                     shape = (n_users, n_items))

    return rate_matrix, users, items, encoder

In [ ]:
rating_matrix, users, items, encoder_dict = create_rate_matrix(ratings)

Train dataset size is 80669, test dataset size is 19317
There are 610 users and 8951 products in dataset.


<ipython-input-32-34e61254fc8a>:13: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  split_point = np.int(np.round(df.shape[0] * split_ratio))


In [ ]:
tfidf = TfidfVectorizer()
movie_train_tfidf = tfidf.fit_transform(movie_genres)
movie_train_tfidf = pd.DataFrame(movie_train_tfidf.toarray(), columns=tfidf.get_feature_names_out())
movies = movies.drop(columns=['genres'])
movies = pd.concat([movies, movie_train_tfidf], axis=1)

In [ ]:
movies['movieId'] = movies['movieId'].apply(lambda x:
                                                      'other' if x not in encoder_dict['movieId'].classes_
                                                      else x)

movies = movies[movies['movieId'] != 'other']
movies['movieId'] = encoder_dict['movieId'].transform(movies['movieId'].values)

<ipython-input-35-0388b0d6620c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['movieId'] = encoder_dict['movieId'].transform(movies['movieId'].values)


In [ ]:
columns = movies.columns.to_list()
columns.remove('movieId')

In [ ]:
def generate_feature_list(df, columns):
    '''
    Generate the list of features of corresponding columns to list
    In order to fit the lightdm Dataset
    '''
    features = df[columns].apply(
        lambda x: ','.join(x.map(str)), axis = 1)
    features = features.str.split(',')
    features = features.apply(pd.Series).stack().reset_index(drop = True)
    return features

In [ ]:
def prepare_item_features(df, columns, id_col_name):
    '''
    Prepare the corresponding feature formats for
    the lightdm.dataset's build_item_features function
    '''
    features = df[columns].apply(
            lambda x: ','.join(x.map(str)), axis = 1)
    features = features.str.split(',')
    features = list(zip(df[id_col_name], features))
    return features

In [ ]:
dataset = light.Dataset()
fitting_item_features = generate_feature_list(movies, columns)
lightdm_features = prepare_item_features(movies, columns, 'movieId')

dataset.fit(users, items, item_features = fitting_item_features)
item_feature = dataset.build_item_features(lightdm_features,
                                            normalize = True)

In [ ]:
model = LightFM()
model.fit(rating_matrix['train'], epochs=10)

train_precision = precision_at_k(model, rating_matrix['train'], k=10).mean()
test_precision = precision_at_k(model, rating_matrix['test'], k=10, train_interactions=rating_matrix['train']).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.36, test 0.16.


In [ ]:
movies_title = np.array(movies['title'])
movies_title

array(['Toy Story (1995)', 'Jumanji (1995)', 'Grumpier Old Men (1995)',
       ..., 'Black Butler: Book of the Atlantic (2017)',
       'No Game No Life: Zero (2017)', 'Flint (2017)'], dtype=object)

In [ ]:
def sample_recommendation(model, data, user_ids):
    n_users, n_items = data['train'].shape
    for user_id in user_ids:
        known_positives = movies_title[data['train'].tocsr()
                          [user_id].indices]

        scores = model.predict(user_id, np.arange(n_items))

        top_items = movies_title[np.argsort(-scores)]

        print(f'Для пользователей с ID {user_id} рекомендуются следующие фильмы:')
        print('--------------')

        for x in top_items[:10]:
            print("        %s" % x)
        print()

In [ ]:
userId = [36, 600]
sample_recommendation(model, rating_matrix, userId)

Для пользователей с ID 36 рекомендуются следующие фильмы:
--------------
        Shawshank Redemption, The (1994)
        Forrest Gump (1994)
        Pulp Fiction (1994)
        Silence of the Lambs, The (1991)
        Matrix, The (1999)
        Braveheart (1995)
        Jurassic Park (1993)
        Schindler's List (1993)
        Star Wars: Episode IV - A New Hope (1977)
        Terminator 2: Judgment Day (1991)

Для пользователей с ID 600 рекомендуются следующие фильмы:
--------------
        Shawshank Redemption, The (1994)
        Forrest Gump (1994)
        Pulp Fiction (1994)
        Silence of the Lambs, The (1991)
        Matrix, The (1999)
        Braveheart (1995)
        Jurassic Park (1993)
        Schindler's List (1993)
        Star Wars: Episode IV - A New Hope (1977)
        Terminator 2: Judgment Day (1991)



Вывод:

В задании были построены соедующие рекомендательные модели на основе:

 1. SVD библиотеки surprise
 2. content-based с применением TF-IDF и метода NearestNeighbors
 3. гибридной модели из двух предыдущих
 4. библиотеки Lightfm